In [1]:
%run 1-0mmyolo-common.ipynb

/bin/bash: nvcc: command not found
gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

/bin/bash: nvidia-smi: command not found
/work/van-speech-nlp/jindaznb/j-vis
System Prefix: /work/van-speech-nlp/jindaznb/visenv
HOME: /work/van-speech-nlp/jindaznb/j-vis
2.0.0+cu117 False
3.3.0
11.7
GCC 9.3


# Write custom Config file

In [2]:
# %cd {HOME}/mmyolo
%cd {HOME}/mmdetection

/work/van-speech-nlp/jindaznb/j-vis/mmdetection


In [3]:
model_folder="grid_rcnn"
custom_model_name="custom-grid-rcnn_x101-64x4d_fpn_gn-head_2x_coco"
CUSTOM_CONFIG_PATH = f"{HOME}/mmdetection/configs/{model_folder}/{custom_model_name}.py"

CUSTOM_CONFIG = f"""
# The new config inherits a base config to highlight the necessary modification
_base_ = 'grid-rcnn_x101-64x4d_fpn_gn-head_2x_coco.py'

# ========================Frequently modified parameters======================

dataset_type = 'COCODataset'
max_epochs = 300
classes =  ('Alive Tree', 'Beetle-Fire Tree', 'Dead Tree', 'Debris') 

data_root = '{dataset_location}'

data = dict(
    samples_per_gpu=8,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        img_prefix='train/',
        classes=classes,
        ann_file='train/annotation_coco.json'),
    val=dict(
        type=dataset_type,
        img_prefix='valid/',
        classes=classes,
        ann_file='valid/_annotations.coco.json'),
    test=dict(
        type=dataset_type,
        img_prefix='test/',
        classes=classes,
        ann_file='test/_annotations.coco.json'))

default_hooks = dict(
    checkpoint=dict(
        type="CheckpointHook",
        save_best="coco/bbox_mAP_50",
        rule="greater",
        max_keep_ckpts=10,
    ),
    early_stopping=dict(
        type="EarlyStoppingHook",
        monitor="coco/bbox_mAP_50",
        patience=10,
        min_delta=0.001
    ),
)

train_cfg=dict(
    max_epochs=max_epochs
)

# -----data related-----
img_scale = (1024, 1024)  # width, height
# ratio range for random resize
random_resize_ratio_range = (0.1, 2.0)
# Cached images number in mosaic
mosaic_max_cached_images = 40
# Number of cached images in mixupep
mixup_max_cached_images = 20
# Batch size of a single GPU during validation
val_batch_size_per_gpu = 32
# Worker to pre-fetch data for each single GPU during validation
val_num_workers = 3

# Config of batch shapes. Only on val.
batch_shapes_cfg = dict(
    type='BatchShapePolicy',
    batch_size=val_batch_size_per_gpu,
    img_size=img_scale[0],
    size_divisor=32,
    extra_pad_ratio=0.5)

# -----model related-----
# The scaling factor that controls the depth of the network structure
deepen_factor = 0.67
# The scaling factor that controls the width of the network structure
widen_factor = 0.75
# Strides of multi-scale prior box
strides = [8, 16, 32]

norm_cfg = dict(type='BN')  # Normalization config

# -----train val related-----
lr_start_factor = 1.0e-5
dsl_topk = 13  # Number of bbox selected in each level
loss_cls_weight = 1.0
loss_bbox_weight = 2.0
qfl_beta = 2.0  # beta of QualityFocalLoss
weight_decay = 0.05

# Save model checkpoint and validation intervals
save_checkpoint_intervals = 10
# validation intervals in stage 2
val_interval_stage2 = 1
# The maximum checkpoints to keep.
max_keep_ckpts = 3
# single-scale training is recommended to
# be turned on, which can speed up training.
env_cfg = dict(cudnn_benchmark=True)


train_dataloader = dict(
    batch_size=4,
)

val_dataloader = dict(
    batch_size=4,
)

test_dataloader = dict(
    batch_size=4,
)

"""
with open(CUSTOM_CONFIG_PATH, 'w') as file:
    file.write(CUSTOM_CONFIG)
print(CUSTOM_CONFIG_PATH)

/work/van-speech-nlp/jindaznb/j-vis/mmdetection/configs/grid_rcnn/custom-grid-rcnn_x101-64x4d_fpn_gn-head_2x_coco.py


In [4]:
# # !python tools/train.py configs/ssd/costom_ssd512.py --resume | tee ../OUT_ssd.txt
# !python tools/train.py configs/grid_rcnn/custom-grid-rcnn_x101-64x4d_fpn_gn-head_2x_coco.py --resume | tee ../OUT_gridrcnn.txt

In [4]:
CONFIDENCE_THRESHOLD = 0.5
NMS_IOU_THRESHOLD = 0.5

ds = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset_location}/test",
    annotations_path=f"{dataset_location}/test/_annotations.coco.json",
)

images = list(ds.images.values())
CUSTOM_WEIGHTS_PATH = glob.glob(f"{HOME}/mmdetection/work_dirs/{custom_model_name}/best_coco_bbox_mAP_50_epoch_*.pth")[-1]
model = init_detector(CUSTOM_CONFIG_PATH, CUSTOM_WEIGHTS_PATH, device=DEVICE)

def callback(image: np.ndarray) -> sv.Detections:
    result = inference_detector(model, image)
    detections = sv.Detections.from_mmdetection(result)
    return detections[detections.confidence > CONFIDENCE_THRESHOLD].with_nms(threshold=NMS_IOU_THRESHOLD)


mean_average_precision = sv.MeanAveragePrecision.benchmark(
    dataset = ds,
    callback = callback
)

print('mAP:', mean_average_precision.map50_95)

Loads checkpoint by local backend from path: /work/van-speech-nlp/jindaznb/j-vis/mmdetection/work_dirs/custom-grid-rcnn_x101-64x4d_fpn_gn-head_2x_coco/best_coco_bbox_mAP_50_epoch_18.pth
mAP: 0.24856195350890542


In [ ]:
ds = sv.DetectionDataset.from_coco(
    images_directory_path=f"{dataset_location}/test",
    annotations_path=f"{dataset_location}/test/_annotations.coco.json",
)

images = list(ds.images.values())
CUSTOM_WEIGHTS_PATH = glob.glob(f"{HOME}/mmdetection/work_dirs/{custom_model_name}/best_coco_bbox_mAP_50_epoch_*.pth")[-1]
model = init_detector(CUSTOM_CONFIG_PATH, CUSTOM_WEIGHTS_PATH, device=DEVICE)

best_map, best_params = optimize_params(ds, model, param_space)
print('Best mAP:', best_map)
print('Best Parameters:', best_params)

Loads checkpoint by local backend from path: /work/van-speech-nlp/jindaznb/j-vis/mmdetection/work_dirs/custom-grid-rcnn_x101-64x4d_fpn_gn-head_2x_coco/best_coco_bbox_mAP_50_epoch_18.pth
|   iter    |  target   | confid... | nms_io... |
-------------------------------------------------
| 1         | 0.2482    | 0.4371    | 0.9556    |
| 2         | 0.2696    | 0.7588    | 0.6388    |
| 3         | 0.2371    | 0.2404    | 0.2404    |
| 4         | 0.2305    | 0.1523    | 0.8796    |
| 5         | 0.2542    | 0.641     | 0.7373    |
| 6         | 0.2993    | 0.894     | 0.6594    |
| 7         | 0.0       | 1.0       | 0.7853    |
| 8         | 0.2332    | 0.1912    | 0.5695    |
| 9         | 0.2749    | 0.8469    | 0.6521    |
| 10        | 0.2967    | 0.918     | 0.57      |
| 11        | 0.2305    | 0.1292    | 0.2937    |
| 12        | 0.2485    | 0.4976    | 0.8419    |
| 13        | 0.2774    | 0.8091    | 0.4982    |
| 14        | 0.2914    | 0.9404    | 0.4315    |
| 15        | 